# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# Python systrem imports
from itertools import product
from collections.abc import Sequence

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt
import sdmxabs as sa
from sdmxabs import MatchType as Mt
from mgplot import (
    series_growth_plot_finalise, 
    line_plot_finalise, 
    finalise_plot,
    set_chart_dir,
    clear_chart_dir,
)

# display charts in this notebook
SHOW_CHARTS = False

# save charts in this notebook
CHART_DIR = "./CHARTS/Household-Spending/"
set_chart_dir(CHART_DIR)
clear_chart_dir()

## Exploratory

In [2]:
f = pd.DataFrame(sa.data_flows()).T
f[f.name.str.contains("Household Spending")]

,agencyID,version,isFinal,name
COVID_HS,ABS,1.0.0,true,"Household Impacts of COVID-19 Survey, Househol..."
HSI_M,ABS,1.4.0,true,Monthly Household Spending Indicator
HSI_Q,ABS,1.2.0,true,Quarterly Household Spending Indicator


In [3]:
pd.DataFrame(sa.data_dimensions("HSI_M")).T

,position,id,version,agencyID,package,class
MEASURE,1,CL_HSI_MEASURE,1.0.0,ABS,codelist,Codelist
CATEGORY,2,CL_HSI_CATEGORY,1.0.0,ABS,codelist,Codelist
PRICE_ADJUSTMENT,3,CL_PRICE_ADJUSTMENT,1.0.0,ABS,codelist,Codelist
TSEST,4,CL_TSEST,1.0.0,ABS,codelist,Codelist
STATE,5,CL_STATE,1.0.0,ABS,codelist,Codelist
FREQ,6,CL_FREQ,1.0.0,ABS,codelist,Codelist
UNIT_MEASURE,NaN,CL_UNIT_MEASURE,1.0.0,ABS,codelist,Codelist
UNIT_MULT,NaN,CL_UNIT_MULT,1.0.0,ABS,codelist,Codelist
OBS_STATUS,NaN,CL_OBS_STATUS,1.0.0,ABS,codelist,Codelist
OBS_COMMENT,NaN,NaN,NaN,NaN,NaN,NaN


## Headlines

In [4]:
def plot_group(
        categories: Sequence[str],
        measures: Sequence[str],
        regions: Sequence[str],
        seasonality: Sequence[str],
) -> None:

    flow_id = "HSI_M"

    for category, measure, tsest, state in product(categories, measures, seasonality, regions):
        # fetch data for this category
        selection = [
            (category, "CATEGORY", Mt.EXACT),
            (measure, "MEASURE", Mt.EXACT),
            (tsest, "TSEST", Mt.EXACT),
            (state, "STATE", Mt.EXACT),
        ]
        data, meta = sa.fetch_selection(flow_id, selection)

        # Check if we have the expected metadata
        if len(meta) != 1:
            print(f"Skipping {category} - {measure} - {state} - {tsest}. "
                  f"Expected 1 metadata, got {len(meta)=}.")
            continue

        # plot
        ylabels = sa.measure_names(meta)
        ydata, ylabel = sa.recalibrate(data, ylabels)
        
        line_plot_finalise(
            ydata,
            annotate=True,
            ylabel = ylabel.iloc[0],
            title = f"{measure} - {category} - {state}",
            lfooter = f"{state}. Current prices. Monthly data. {tsest}. ",
            rfooter = f"ABS {meta['DATAFLOW'].iloc[0]}",
            tag=f"{state}-{tsest}",
            show=SHOW_CHARTS,
        )

        series_growth_plot_finalise(
            ydata[data.columns[0]],
            plot_from=-16,
            title = f"{measure} - {category} - {state}",
            lfooter = f"{state}. Current prices. Monthly data. {tsest}. ",
            rfooter = f"ABS {meta['DATAFLOW'].iloc[0]}",
            tag=f"{tsest}growth",
            show=SHOW_CHARTS,
        )

    


def plot_headlines() -> None:
    # top level categories
    cats = pd.DataFrame(sa.code_list_for_dim("HSI_M", "CATEGORY")).T
    cats = cats[cats["parent"].isna() | (cats["parent"] == "TOT")]
    categories = cats["name"].tolist()

    # other selection parameters
    measures = ["Household spending"]
    seasonality = ["Seasonally Adjusted"]
    regions = ["Australia"]

    plot_group(
        categories=categories,
        measures=measures,
        regions=regions,
        seasonality=seasonality,
    )


plot_headlines()

Error fetching HSI_M with dimensions {'CATEGORY': '45+450+4500', 'MEASURE': '7', 'TSEST': '20', 'STATE': 'AUS'}: Cached file not available: cache--a7c26327bb9a167abe57aea4c9df7e18db3444e99d055d84ad1c19834bcacac8--7.45+450+4500..20.AUS.
Skipping Communications - Household spending - Australia - Seasonally Adjusted. Expected 1 metadata, got len(meta)=0.
Error fetching HSI_M with dimensions {'CATEGORY': '55+550+5500', 'MEASURE': '7', 'TSEST': '20', 'STATE': 'AUS'}: Cached file not available: cache--7667b6214c650a9fa312f10cf845f6974bebf5681d1c5beee55ab570ae6a3046--7.55+550+5500..20.AUS.
Skipping Education services - Household spending - Australia - Seasonally Adjusted. Expected 1 metadata, got len(meta)=0.
Error fetching HSI_M with dimensions {'CATEGORY': '25', 'MEASURE': '7', 'TSEST': '20', 'STATE': 'AUS'}: Cached file not available: cache--8330f3955be1418c6097c89153a6cac8b000fc3b78a0d3082a7f939c177e4dca--7.25..20.AUS.
Skipping Housing, water, electricity, gas and other fuel - Household s

## States

In [5]:
def plot_states() -> None:
    # top level categories
    regs = pd.DataFrame(sa.code_list_for_dim("HSI_M", "STATE")).T
    regs = regs[regs["parent"].notna() | (regs["parent"] == "TOT")]
    regions = regs["name"].tolist()[:-1]

    # other selection parameters
    categories = ["Total"]
    measures = ["Household spending"]
    seasonality = ["Seasonally Adjusted"]

    plot_group(
        categories=categories,
        measures=measures,
        regions=regions,
        seasonality=seasonality,
    )

    bag_of_states = {}
    for state in regions:
        p = parameters={"startPeriod": "2024-Q1"}
        pop_data, _metapop = sa.fetch_state_pop(state, parameters=p)
        pop = pop_data.iloc[-1, 0] * 1000
        selection = [
            (categories[0], "CATEGORY", Mt.EXACT),
            (measures[0], "MEASURE", Mt.EXACT),
            (seasonality[0], "TSEST", Mt.EXACT),
            (state, "STATE", Mt.EXACT),
        ]
        state_data, _state_meta = sa.fetch_selection("HSI_M", selection)
        spending = state_data.iloc[-1, 0] * 1_000_000
        bag_of_states[state] = spending / pop


    plotable = pd.Series(bag_of_states, name="Monthly Household Spending per Capita")
    ax = plotable.sort_values().plot(kind='barh')
    finalise_plot(
        ax,
        title="Monthly Household Spending per Capita by State",
        ylabel=None,
        xlabel="Spending per Capita (AUD)",
        lfooter=f"Australia, Month:{state_data.index[-1]}. Seasonally adjusted. Current prices. ",
        rfooter="Calculated using ABS data",
        show=SHOW_CHARTS,
    )

plot_states()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-07-14 20:35:17

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot    : 0.2.6
sdmxabs   : 0.1.8a3
pandas    : 2.3.1
matplotlib: 3.10.3

Watermark: 2.5.0



In [7]:
print("Done")

Done
